# Comparing ndmg branches

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

## Metadata

- This analysis uses the KKI2009 data <a href="http://mrneurodata.s3-website-us-east-1.amazonaws.com/KKI2009/" target="_blank">here</a>.
- Docker containers were used.
    - <a href="https://hub.docker.com/r/bids/ndmg/" target="_blank">master</a>
    - <a href="https://hub.docker.com/r/bids/ndmg/" target="_blank">eric's branch</a>.
- I used the outputs in 
    - **master**: /graphs/ (.edgelist)
    - **eric**: /dwi/roi-connectomes (csv)

## Plan

1. Eric
    - Load data as np array
    - Convert to adjacency matrix
2. Master
    - Load data as np array
    - Convert to adjacency matrixc
3. Define a function that takes a file location as input and returns an adjacency matrix as output
    - Check whether file location is from master or eric
    - Possibly needs to do different things depending on which branch it's taking data from
4. For each pair of supposedly-equivalent graphs, take the difference
5. Take the norm of the above
6. Use seaborn to plot this for every parcellation
7. Equivalent analysis for group-level data

## Load data

In [2]:
# Show the data I ran on in a tree structure. Ignore the \xa0\xa0's. Ran on 3 of the subjects from the KKI2009 dataset.
%system tree -d -L 3 /data/aloftus/KKI2009

['/data/aloftus/KKI2009',
 '├── sub-113',
 '│\xa0\xa0 ├── ses-1',
 '│\xa0\xa0 │\xa0\xa0 ├── anat',
 '│\xa0\xa0 │\xa0\xa0 ├── dwi',
 '│\xa0\xa0 │\xa0\xa0 └── func',
 '│\xa0\xa0 └── ses-2',
 '│\xa0\xa0     ├── anat',
 '│\xa0\xa0     ├── dwi',
 '│\xa0\xa0     └── func',
 '├── sub-127',
 '│\xa0\xa0 ├── ses-1',
 '│\xa0\xa0 │\xa0\xa0 ├── anat',
 '│\xa0\xa0 │\xa0\xa0 ├── dwi',
 '│\xa0\xa0 │\xa0\xa0 └── func',
 '│\xa0\xa0 └── ses-2',
 '│\xa0\xa0     ├── anat',
 '│\xa0\xa0     ├── dwi',
 '│\xa0\xa0     └── func',
 '└── sub-142',
 '    ├── ses-1',
 '    │\xa0\xa0 ├── anat',
 '    │\xa0\xa0 ├── dwi',
 '    │\xa0\xa0 └── func',
 '    └── ses-2',
 '        ├── anat',
 '        ├── dwi',
 '        └── func',
 '',
 '27 directories']

### Grab output for one subject, one session

- Using JHU, sub-113, ses-1 for testing purposes

In [3]:
# Save directory data
eric_graphs = r'/data/aloftus/KKI-output/eric-branch/participant/dwi/roi-connectomes/JHU_res-1x1x1/'
master_graphs = r'/data/aloftus/KKI-output/master/graphs/JHU/'

In [4]:
# Show output csv files. Arbitrarily using ses-1 of sub-113 as testing data.
print(os.listdir(eric_graphs), '\n')
print(os.listdir(master_graphs))

['sub-127_ses-2_dwi_JHU_res-1x1x1_measure-spatial-ds._elist.csv', 'sub-127_ses-1_dwi_JHU_res-1x1x1_measure-spatial-ds._elist.csv', 'sub-142_ses-1_dwi_JHU_res-1x1x1_measure-spatial-ds._elist.csv', 'sub-113_ses-2_dwi_JHU_res-1x1x1_measure-spatial-ds._elist.csv', 'sub-142_ses-2_dwi_JHU_res-1x1x1_measure-spatial-ds._elist.csv', 'sub-113_ses-1_dwi_JHU_res-1x1x1_measure-spatial-ds._elist.csv'] 

['sub-113_ses-2_dwi_JHU.edgelist', 'sub-142_ses-2_dwi_JHU.edgelist', 'sub-127_ses-2_dwi_JHU.edgelist', 'sub-127_ses-1_dwi_JHU.edgelist', 'sub-113_ses-1_dwi_JHU.edgelist', 'sub-142_ses-1_dwi_JHU.edgelist']


In [19]:
# Get networkX graph for eric's sub-113-ses-1-JHU
eric_edgelist_location = eric_graphs + 'sub-113_ses-1_dwi_JHU_res-1x1x1_measure-spatial-ds._elist.csv'
eric_nx = nx.read_weighted_edgelist(eric_edgelist_location, delimiter=',')

# Get networkX graph for masters's sub-113-ses-1-JHU
master_edgelist_location = master_graphs + 'sub-113_ses-1_dwi_JHU.edgelist'
master_nx = nx.read_weighted_edgelist(master_edgelist_location)

In [17]:
eric_nx.nodes()

NodeView(('37', '33', '41', '34', '29', '15', '4', '42', '16', '22', '36', '12', '8', '35', '25', '31', '43', '47', '32', '1', '27', '2', '7', '30', '3', '23', '26', '24', '9', '45', '17', '19', '18', '11', '14', '39', '46', '6', '21', '28', '13', '20', '48', '5', '10', '40', '38', '44'))

In [20]:
master_nx.nodes()

NodeView(('37', '33', '41', '34', '29', '15', '4', '42', '16', '22', '36', '12', '8', '35', '25', '31', '43', '32', '1', '48', '27', '2', '7', '30', '3', '23', '26', '24', '9', '45', '17', '21', '18', '11', '14', '39', '46', '6', '19', '28', '13', '20', '47', '5', '10', '40', '38', '44'))

In [23]:
nx.adj_matrix(eric_nx)

<48x48 sparse matrix of type '<class 'numpy.float64'>'
	with 634 stored elements in Compressed Sparse Row format>

## Function to load and compare graphs

Function should: 
1. Load a pair of graphs
2. Ensure they're in a consistent and sensible format (e.g., adjacency matrix)
3. Subtract one graph from the other
4. Take the norm of the result

In [11]:
def load_and_compare(master: 'path', eric: 'path'):
    
    # Load a pair of graphs
    graph_pair = [nx.read_weighted_edgelist(master), nx.read_weighted_edgelist(eric, delimiter=',')]
    
    # Move each graph to adjacency matrix
    graph_pair = [nx.adj]
    
    # Subtract one graph from the other
    # Take norm of result
    pass

## nx playground, to be removed

In [9]:
G = nx.Graph()
G.add_edge(24, 24, weight=4150)
G.add_edge(25, 25, weight=2300)

nx.to_numpy_array(G)  # numpy array is not organized the same way as the edgelist file.
nx.to_edgelist(G)

EdgeDataView([(24, 24, {'weight': 4150}), (25, 25, {'weight': 2300})])

### Master output